In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import csv
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

w = SummaryWriter(log_dir='logs', comment='Net')

class DigitSet(Dataset) :
    def __init__(self, path, model = 'train') :
        super().__init__()
        self.model = model
        with open(path) as file :
            data_csv = list(csv.reader(file))#读取数据
            data = np.array(data_csv[1 : ]).astype(np.float32)
        if self.model == 'test' :
            self.data = torch.FloatTensor(data.reshape(len(data), 1, 28, 28))
        else :
            target = data[ : , 0]
            data = data[ : , 1 : ]
            train_index = []
            val_index = []
            for i in range(data.shape[0]) :
                if i % 10 != 0 :
                    train_index.append(i)
                else :
                    val_index.append(i)
            if model == 'train' :
                self.data = torch.FloatTensor(data[train_index, : ].reshape(len(train_index), 1, 28, 28))
                #numpy转化为tenso对于图片而言需要进行轴转换，因为numpy表示图片为(H, W, C)，而tensor是(C, H, W)
                self.target = torch.LongTensor(target[train_index])
            else :
                self.data = torch.FloatTensor(data[val_index, : ].reshape(len(val_index), 1, 28, 28))
                self.target = torch.LongTensor(target[val_index])
        self.data = self.data / 255
        self.dim = data.shape[1]
    def __getitem__(self, item) :
        if self.model == 'train' or self.model == 'val' :
            return self.data[item], self.target[item]
        else :
            return self.data[item]
    def __len__(self) :
        return len(self.data)
class Classifier(nn.Module) :
    def __init__(self) :
        super(Classifier, self).__init__()
        self.CNN = nn.Sequential(
            nn.Conv2d(1, 16, 3, 1, 1), #卷积层
            nn.BatchNorm2d(16),#归一化
            nn.ReLU(),#激活函数
            nn.MaxPool2d(2, 2, 0), #池化层，缩小图像

            nn.Conv2d(16, 32, 3, 1, 1),  
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  
        )
        self.net = nn.Sequential(
            nn.Linear(32 * 7 * 7, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10),)
    def forward(self, x) :
        output = self.CNN(x)
        output = output.view(output.size(0), -1)
        return F.softmax(self.net(output))

# def weights_init(m) :
#     if isinstance(m, nn.Linear) :
#         nn.init.uniform_(m.weight)
#         nn.init.constant_(m.bias, 0.0)
#     elif isinstance(m, nn.Conv2d) :
#         nn.init.normal_(m.weight, mean = 0, std = 0.5)
#         nn.init.constant_(m.bias, 0.0)
    
Batch_size = 64
train_set = DigitSet('train.csv', model = 'train')
val_set = DigitSet('train.csv', model = 'val')
test_set = DigitSet('test.csv', model = 'test')

train_loader = DataLoader(train_set, batch_size=Batch_size, shuffle=True, drop_last=False)
val_loader = DataLoader(val_set, batch_size=Batch_size,shuffle=False, drop_last=False)
test_loader = DataLoader(test_set, batch_size=Batch_size, shuffle=False, drop_last=False)

model = Classifier().cuda()
# model.apply(weights_init)
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()
max_epoch = 20
train_losses = []
val_acces = []

for epoch in range(max_epoch) :
    train_loss = 0.0
    train_acc = 0.0
    min_loss = 1000
    model.train()
    for x, label in train_loader :
        x, label = x.cuda(), label.cuda()
        optimizer.zero_grad()#梯度清零
        
        output = model(x)#前向传播
        loss = criterion(output, label)
        
        train_loss += loss.item()
        
        loss.backward()#反向传播
        optimizer.step()#参数更新
        _, pred = output.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / x.shape[0]
        #train_acces.append(acc)
        train_acc += acc
    total_loss = []
    with torch.no_grad() :
        model.eval()
        for y, data in enumerate(val_loader) :
            x, label = data[0].cuda(), data[1].cuda()
            output = model(x)
            total_loss.append(criterion(output, label))
            dev_loss = sum(total_loss) / len(total_loss)
    if dev_loss < min_loss :
        min_loss = dev_loss
        torch.save(model.state_dict(), 'mymodel.pth')#保存最好模型
    print('epoch : {:02d}, train_loss : {}, train_acc : {}'.format(epoch + 1, 
                                                               train_loss / len(train_loader), 
                                                               train_acc  / len(train_loader)))
    train_losses.append(train_loss / len(train_loader))
    val_acces.append(train_acc  / len(train_loader))
    w.add_scalar('train_losses', train_loss / len(train_loader), epoch)#可视化训练损失
    w.add_scalar('val_acc', train_acc / len(train_loader), epoch)#可视化验证集正确率
del model
bestmodel = Classifier().cuda()
ckpt = torch.load('mymodel.pth', map_location="cpu")
bestmodel.load_state_dict(ckpt)
with torch.no_grad() :
    bestmodel.eval()
    output = []
    for x in test_loader :
        x = x.cuda()
        out = bestmodel(x)
        _, pred = torch.max(out, 1)
        for i in pred.cpu().numpy():
            output.append(i)
    #output = torch.cat(output, dim = 0).data.cpu().numpy()
with open('prediction.csv', 'w') as file :
    writer = csv.writer(file)
    writer.writerow(['ImageId', 'Label'])
    for i, pred in enumerate(output) :
        writer.writerow([i, pred])
# epoch = list(range(max_epoch))
# plt.figure('figure one')
# plt.plot(epoch, train_losses, label = 'Train_loss')
# plt.xlabel('x轴')
# plt.ylabel('y轴')
# plt.title('Loss')
# plt.figure('figure two')
# plt.plot(epoch, val_acces, label = 'Val_acc')
# plt.xlabel('x轴')
# plt.ylabel('y轴')
# plt.title('Acc')
# plt.legend()
# plt.show()

<ipython-input-5-32aea4cf34de>:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.net(output))


epoch : 01, train_loss : 1.8135494082510573, train_acc : 0.7033047800338409
epoch : 02, train_loss : 1.6033565376817474, train_acc : 0.8712087563451776
epoch : 03, train_loss : 1.5755864476592811, train_acc : 0.8949978849407784
epoch : 04, train_loss : 1.4992643098540717, train_acc : 0.9732127749576989
epoch : 05, train_loss : 1.4890364214853586, train_acc : 0.9787700930626058
epoch : 06, train_loss : 1.4845159471337566, train_acc : 0.981868654822335
epoch : 07, train_loss : 1.4812717117028793, train_acc : 0.984665820642978
epoch : 08, train_loss : 1.4787657369816969, train_acc : 0.986410744500846
epoch : 09, train_loss : 1.4769414695588061, train_acc : 0.9880234771573604
epoch : 10, train_loss : 1.4756469724545422, train_acc : 0.9886051184433164
epoch : 11, train_loss : 1.4741797055891523, train_acc : 0.9901385363790186
epoch : 12, train_loss : 1.473053337555448, train_acc : 0.9909686971235195
epoch : 13, train_loss : 1.4719505969642988, train_acc : 0.9917882825719121
epoch : 14, trai